In [9]:
import pandas as pd
import pickle

with open('p5r_data.pkl', 'rb') as file:
    data = pickle.load(file)

df_persona = pd.read_csv('p5r_人格面具.csv', index_col=0)
df_compose = pd.read_csv('p5r_合成总表.csv')
df_item = pd.read_csv('p5r_道具图鉴.csv', index_col=0)

persona_level = dict(zip(df_persona['中文名'].tolist(), df_persona['等级'].tolist()))
df_compose['Level'] = df_compose['Result'].map(persona_level)
df_compose['Lv.1'] = df_compose['Name.1'].map(persona_level).fillna(df_compose['Lv.1']).fillna(-1).astype(int)

df_item['备注'] = df_item['备注'].str.split('/').str[-1]
df_item['Lv'] = df_item['备注'].map(persona_level).fillna(-1).astype(int)

df_compose.head()

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result,Level
0,6561.0,愚者,8,背负怪,愚者,11,俄耳普斯F,亚森,1
1,7077.0,愚者,8,背负怪,愚者,13,俄耳普斯·贼神F,亚森,1
2,6084.0,魔术师,5,猫妖精,魔术师,11,杰克霜精,皮克希,2
3,6084.0,魔术师,5,猫妖精,魔术师,11,杰克霜精,皮克希,2
4,6741.0,恋人,6,幸魂,恋人,13,天钿女命,皮克希,2


In [30]:
# 查询反向合成表
df_compose.loc[df_compose['Result'].str.contains('')]

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result,Level
284,5325.0,倒悬者,9,花魄,恋人,2,皮克希,奇魂,12
285,5517.0,隐者,4,双角兽,教皇,9,比利士,奇魂,12
286,5568.0,月,7,女梦魔,节制,7,玄武,奇魂,12
287,5781.0,力量,6,凯尔皮,正义,9,天使,奇魂,12
288,5781.0,倒悬者,9,花魄,恋人,6,幸魂,奇魂,12
289,5856.0,星星,11,木灵,死神,3,曼陀罗,奇魂,12
290,5940.0,月,7,女梦魔,倒悬者,9,花魄,奇魂,12
291,6312.0,隐者,9,克鲁波克鲁,教皇,9,比利士,奇魂,12
292,6384.0,月,7,女梦魔,节制,11,木叶天狗,奇魂,12
293,6570.0,倒悬者,14,犬神,恋人,2,皮克希,奇魂,12


In [7]:
# 查询技能，可尝试多个idx
name = '杰克霜精'
print(f'可选表：{list(range(len(data[name])))}')
data[name][2].iloc[4:, 1:6]

可选表：[0, 1, 2, 3, 4]


,1,2,3,4,5
4,技能,消耗,解锁等级,解锁等级,描述
5,布芙,4 SP,自带,自带,对1名敌人造成冰冻属性的小量伤害，罕见使敌人陷入冻结
6,消除冰冻防御,6 SP,自带,自带,消除所有敌人的冰冻耐性，无法消除防冰之壁
7,拜斯堤,4 SP,自带,自带,治疗1名伙伴的燃烧、冻结、触电状态
8,玛哈布芙,10 SP,12,12,对所有敌人造成冰冻属性的小量伤害，罕见使敌人陷入冻结
9,拉坤达,8 SP,13,13,在3回合间，使1名敌人的防御力下降
10,冻结几率UP,NaN,15,15,NaN


In [10]:
# 查询任意种族，等级面具
df_persona.loc[df_persona['中文名'].str.contains('八')]
# df_persona.loc[df_persona['等级'] == 64]
# df_persona.loc[df_persona['塔罗牌'] == '审判']

,中文名,等级,英文名,日文名,塔罗牌,物,枪,火,冰,电,风,念,核,祝,咒
160,八咫乌,57,Yatagarasu,ヤタガラス,顾问官,-,-,-,-,-,耐,-,-,无,弱
176,八岐大蛇,64,Yamata-no-Orochi,ヤマタノオロチ,审判,耐,-,-,吸,-,耐,-,-,-,耐


In [10]:
# 指定等级范围查询一步能合成的面具
name = '基力梅卡拉'
level = "50 - 60"

level_range = level.split(' - ')
name_cond = (df_compose['Name'].str.contains(name)) | (df_compose['Name.1'].str.contains(name))
level_cond = (df_compose['Level'] > int(level_range[0])) & (df_compose['Level'] < int(level_range[1]))
df_compose.loc[name_cond & level_cond]
# df_compose.loc[name_cond & level_cond, ['Result', 'Level']].drop_duplicates('Result')     # 仅显示合成结果及等级

,价格,种族,Lv,Name,种族.1,Lv.1,Name.1,Result,Level
10395,36420.0,月,43,基力梅卡拉,死神,37,地狱天使,御社宫司神,52
10405,38625.0,月,43,基力梅卡拉,女教皇,40,菊理媛,御社宫司神,52
10466,47145.0,月,43,基力梅卡拉,女教皇,50,妙音天女,御社宫司神,52
10493,50052.0,月,43,基力梅卡拉,女教皇,53,丝卡蒂,御社宫司神,52
10505,51057.0,月,43,基力梅卡拉,死神,54,苍白骑士,御社宫司神,52
11324,47145.0,月,43,基力梅卡拉,女皇,50,荼吉尼,诺伦,52
11344,53121.0,月,43,基力梅卡拉,女皇,56,蒂坦妮娅,诺伦,52
11499,49065.0,月,43,基力梅卡拉,命运之轮,52,诺伦,象头神,53
11506,51057.0,信念,54,大国主,月,43,基力梅卡拉,象头神,53
11518,53121.0,月,43,基力梅卡拉,命运之轮,56,亚斯泰利欧斯,象头神,53


In [7]:
from search_route import search_route

start = '大僧正'  # 设置合成起点
end = '爱丽丝'      # 设置合成终点
head_num = 10       # 设置显示个数（按合成成本升序）
max_depth = 3       # 设置搜索最大深度

search_route(start, end, max_depth=max_depth, head_num=head_num)

大僧正 + 猫妖 -> 苍白骑士 + 托尔 -> 奈比洛斯 = 95655.0 --特殊合成-> 爱丽丝
大僧正 + 持国天 -> 苍白骑士 + 托尔 -> 奈比洛斯 = 97620.0 --特殊合成-> 爱丽丝
大僧正 + 睡魔 -> 苍白骑士 + 托尔 -> 奈比洛斯 = 98067.0 --特殊合成-> 爱丽丝
大僧正 + 阴魔罗鬼 -> 菊理媛 + 加百列 -> 奈比洛斯 = 98751.0 --特殊合成-> 爱丽丝
大僧正 + 大天使 -> 百臂巨人 + 库夫林 -> 奈比洛斯 = 99444.0 --特殊合成-> 爱丽丝
大僧正 + 辉夜 -> 菊理媛 + 加百列 -> 奈比洛斯 = 99927.0 --特殊合成-> 爱丽丝
大僧正 + 和魂 -> 能天使 + 圣达芬 -> 奈比洛斯 = 100182.0 --特殊合成-> 爱丽丝
大僧正 + 克伦佐 -> 苍白骑士 + 托尔 -> 奈比洛斯 = 100572.0 --特殊合成-> 爱丽丝
大僧正 + 黑软泥怪 -> 菊理媛 + 加百列 -> 奈比洛斯 = 100623.0 --特殊合成-> 爱丽丝
大僧正 + 佛劳洛斯 -> 百臂巨人 + 库夫林 -> 奈比洛斯 = 101139.0 --特殊合成-> 爱丽丝


In [59]:
# 查询道具图鉴

cond_app = df_item['适用'].str.contains('杏')
cond_type = df_item['类型'].str.contains('近战武器')
cond_all = True
df_item.loc[cond_type & cond_app]

,中文名,日文名,类型,属性,追加效果,获取,适用,备注,Lv
97,燃火鞭,バーンウィップ,近战武器,攻98 命97,低概率附加燃烧,商店售卖,高卷杏,6月21,-1
102,山羊皮鞭,山羊革のウィップ,近战武器,攻86 命99,NaN,商店售卖,高卷杏,5月18,-1
107,裂肤者,スキンリッパー,近战武器,攻68 命97,低机率附加恐惧,商店售卖,高卷杏,5月18,-1
111,分铜锁,分銅鎖,近战武器,攻58 命99,NaN,商店售卖,高卷杏,4月17,-1
115,炽红鞭,ヒートウィップ,近战武器,攻48 命97,低机率附加燃烧,商店售卖,高卷杏,4月17,-1
152,大深渊之鞭,NaN,近战武器,攻320 命90,高机率附加冰冻,警报电刑,高卷杏,邪恶霜精,67
160,缚狼锁,グラウプニル,近战武器,攻312 命99,NaN,商店售卖,高卷杏,1月13,-1
171,炎魔鞭,バルログウィップ,近战武器,攻292 命97,中机率附加燃烧,商店售卖,高卷杏,1月13,-1
179,冰雪女王鞭·改,NaN,近战武器,攻280 命90,在战斗开始时发动斯库卡加,警报电刑,高卷杏,丝卡蒂,53
180,深渊之鞭,NaN,近战武器,攻280 命90,高机率附加冰冻,警报电刑,高卷杏,邪恶霜精,67
